# Mini Python/Interpretation Project for Junior Geophysicist Position

## Quick Warning

Most of my Python scripts were run in a virtual (Conda) environment with all the necessary libraries installed. These scripts also depend on a specific folder structure, so running them directly in this Jupyter Notebook isn’t feasible.

## Data Organization

The first step was to organize the data using Python. In total, I wrote 9 scripts, each containing multiple functions, to tackle this. Essentially, `main.py` calls individual parsers/processors for the various file types in the project. The order of processing is: shapefiles (.shp), TIFFs (.tif, .tiff), GDB files (.gdb), XYZ files (.xyz), ERS files (.ers), CSVs, and finally other files (like .pdfs). All of these scripts, including reusable functions in `utils.py`, are included here.

**utils.py**

**main.py**

**process_shapefile.py**

**process_tiff.py**

**process_gdb.py**

**process_xyz.py**

**process_ers.py**

**process_csv.py**

**process_other.py**

On my system, the script was run in a root directory named "data". Once the main script finishes, the files are moved and reorganized into a new file structure (shown below).

```
    data/
    ├── processed/
        ├── empty_data
        └── other_data
        └── raster_data/
            ├── EPSG_4326
        └── vector_data/
            ├── EPSG_4267
            └── EPSG_4269
            └── EPSG_4326
            └── EPSG_26711
            └── EPSG_26911
        └── summary.txt


```

All vector and raster datasets have their Coordinate Reference Systems (CRS) extracted and are grouped into subfolders by CRS. This helped me quickly understand the different CRSs I was dealing with. Empty files are sent to an "empty_data" folder. TIFFs, ERD, GRD, and MAP files are moved to a "raster_data" folder, while shapefiles, GDBs, and XYZ files go into "vector_data". Non-spatial files (like PDFs) are stored in "other_data". The Geosoft formats (GDBs, GRDs, MAPs) were a bit tricky, and I manually converted them to CSVs or GeoTIFFs using Geosoft Viewer. While the script creates new files during this process, nothing is deleted. CSV conversions make Python processing easier, and these files are stored in their respective directories, along with any associated files from the root directory. The XYZ file processing script is a bit specific to the dataset I had, and while it works, a more flexible version would be better for general use. A summary text file is generated at the end, tracking file types, their total size, empty files found, and the number of folders created.

## Data Cleaning

After organizing the data, I wrote a script to convert everything to EPSG:4326 (WGS84). I haven’t dealt with CRS conversions in a while, so I ignored any potential side effects (probably not best practice), but the reprojected datasets lined up well in QGIS. The `crs_conversion` script looks through all non-EPSG_4326 subdirectories in the "vector_data" folder and converts shapefiles and CSVs to EPSG:4326. The originals are left untouched.

**crs_conversion.py**

I wrote `generate_file_overview.py` to help quickly open shapefiles and CSVs, visualize column histograms, and generate key statistics. This helped spot outliers quickly. For instance, some datasets had undefined values set as -999999 or -9999.9, which were immediately obvious in the histograms. I also wrote `clean_data.py`, which allows users to set upper and lower bounds for data cleaning. It currently only handles numeric data, but expanding it to handle non-numeric data would be straightforward. The scripts, a supporting `util.py`, and a few histograms (pre- and post-cleaning) are included below.

**util.py**

**generate_file_overview.py**

**clean_data.py**

![A histogram showing frequencies of the Apparent Potassium (%) column in the original reno_mag dataset](figures_for_jn/f1.png)

**Figure 1**: A histogram showing frequencies of the Apparent Potassium (%) column in the original `reno_mag` dataset.

![A histogram showing frequencies of the Apparent Potassium (%) column in the cleaned reno_mag dataset](figures_for_jn/f2.png)

**Figure 2**: A histogram showing frequencies of the Apparent Potassium (%) column in the _cleaned_ reno_mag dataset.

## Data Exploration

I attempted to visualize all the data using Python but quickly realized I was in over my head. Rendering multiple data types on a map proved challenging, especially in Folium. The map rendering was slow, layers were messy (see Figure 3), I couldn’t figure out how to add a legend, and nothing was as interactive as I wanted. In the end, I gave up and switched to QGIS, which turned out to be much more efficient.

**visualize_files.py**

![The "interactive" Folium map I created, displaying DNAG magnetic field data, residual magnetic field strength lines and significant deposits](figures_for_jn/f3.png)

**Figure 3**: The "interactive" Folium map I created, displaying DNAG magnetic field data, residual magnetic field strength lines and significant deposits.

After visualizing everything, I focused on a subset of files that I found intriguing. A map, generated in QGIS, is shown below.

![A QGIS-generated map showing the total magnetic field strength (DNAG), the residual magnetic field strength (reno_mag and water_lake_mag), faults, precious metals, significant deposits, and active mining claims](figures_for_jn/f4.png)

**Figure 4**: A QGIS-generated map showing the total magnetic field strength (DNAG), the residual magnetic field strength (reno_mag and water_lake_mag), faults, precious metals, significant deposits, and active mining claims.

What really caught my attention were the positive residual magnetic field anomalies near the Cu-colored precious metal markers on the map’s eastern side. Magnetic minerals can be linked to hydrothermal systems (e.g., magnetite forms as a byproduct of hydrothermal alteration). This might suggest some association with porphyry copper systems, which could explain why the precious metals markers are in this area. Given my interest in the granularity of the reno and walker_lake XYZ files, I also checked out the radiometric data along these lines, and apparent potassium values were relatively high in this zone.

![A QGIS-generated map showing the total magnetic field strength (DNAG), apparent potassium (reno_rad and water_lake_rad), faults, precious metals, significant deposits, and active mining claims](figures_for_jn/f5.png)

**Figure 5**: A QGIS-generated map showing the total magnetic field strength (DNAG), apparent potassium (reno_rad and water_lake_rad), faults, precious metals, significant deposits, and active mining claims.

I wanted to explore Bouguer anomalies in the area (one of the few things I vaguely remember from school). After digging through the provided files, I found the USA_isograv dataset, which seemed to have what I needed. Because point data isn’t as fun as raster data, I wrote a script to convert point data (from CSVs) into Matplotlib heatmaps and then GeoTIFFs (with Rasterio). However, Rasterio kept rotating the heatmaps 90 degrees. After some trial and error, I fixed this by transposing the z_grid and flipping it. The Bouguer anomaly heatmap, along with the point data, can be seen below.

**heatmap_generation_from_csv.py**

![A Matplotlib heatmap for isostatically-corrected Bouguer gravity anomalies](figures_for_jn/f6.png)

**Figure 6**: A Matplotlib heatmap for isostatically-corrected Bouguer gravity anomalies.

![A QGIS-generated map showing the same heatmap (after being converted to a GeoTIFF), faults, precious metals, significant deposits, and active mining claims](figures_for_jn/f7.png)

**Figure 7**: A QGIS-generated map showing the same heatmap (after being converted to a GeoTIFF), faults, precious metals, significant deposits, and active mining claims.

To my untrained eye, this didn't provide much insight with respect to my hydrothermal alteration hypothesis. However, all of the known deposits on the map appear to be associated with similar, but relatively high Bouguer anomalies. Perhaps this is something that could be taken into consideration in the future.

## Advanced Analysis (Clustering)

Excited about the data, I decided to perform K-Means clustering on the combined reno_rad and walker_lake_rad datasets (combined as CSVs). These datasets contained granular info on residual magnetic fields and radiometric data, which I found potentially interesting for exploration. K-Means clustering is easy to use, computationally efficient, and relatively simple to interpret, but it has its drawbacks, especially for geological data. Picking the wrong k can hide key features, and the algorithm assumes clusters are spherical, which doesn’t reflect real-world geology. To help find a good k, I used the Elbow Method, which plots the sum of squared distances between points and their cluster centroids. I settled on k = 5, clustering based on residual magnetic fields, apparent thorium, and apparent potassium values. Below is the script, the elbow plot, and cluster plots.

**k_means_clustering.py**

![A plot generated by the Elbow Method, used to decide on a value for k](figures_for_jn/f8.png)

**Figure 8**: A plot generated by the Elbow Method, used to decide on a value for k.

![Clusters for residual magnetic field vs. apparent thorium values](figures_for_jn/f9a.png)

**Figure 9a**: Clusters for residual magnetic field vs. apparent thorium values.

![Clusters for residual magnetic field vs. apparent potassium values](figures_for_jn/f9b.png)

**Figure 9b**: Clusters for residual magnetic field vs. apparent thorium values.

Clusters were then plotted in QGIS:

![A QGIS-generated map showing clusters generated with K-Means clustering based on residual magnetic field, apparent thorium and apparent potassium values](figures_for_jn/f10.png)

**Figure 10**: A QGIS-generated map showing clusters generated with K-Means clustering based on residual magnetic field, apparent thorium and apparent potassium values.

Interestingly, the Cu-heavy zone corresponds to cluster 1 (blue circles), which aligns with residual magnetic field patterns. Similar patterns were identified in other lines that weren’t as obvious using magnetic data alone. For example, additional faulting was highlighted to the northwest.

## Anomaly Mapping

I decided to try HDBSCAN for anomaly detection, since it was suggested as a potential clustering method in the project instructions. HDBSCAN produces “soft” clusters and adjusts to varying densities, which seemed useful here. I used flight line numbers, longitude, latitude, and residual magnetic fields as inputs. However, most points were flagged as anomalies. I tried tweaking the `min_cluster_size` and `min_samples` parameters but didn’t have much success. The resulting dataset didn’t add much value for mapping potential mineral zones.

**anomaly_detection_hdbscan.py**

![A QGIS-generated map showing anomalies detected using the HDBSCAN method with flight line number, longitude, latitude, and residual magnetic field data used as inputs](figures_for_jn/f11.png)

**Figure 11**: A QGIS-generated map showing anomalies detected using the HDBSCAN method with flight line number, longitude, latitude, and residual magnetic field data used as inputs.

For fun, I gave an alternate method, DBSCAN, a shot. Like K-Means, DBSCAN creates hard clusters, but it uses `eps` (the neighborhood distance) instead of `min_cluster_size`. You can estimate `eps` using a k-distance graph, which is similar to the Elbow Method used in K-Means. DBSCAN works best when data points have a uniform density, which fits the reno and walker_lake datasets well. The anomaly detection here produced more useful results, which matched the K-Means clusters. Below is the DBSCAN script, k-distance graph, and anomaly map.

**anomaly_detection_dbscan.py**

![The k-distance graph used to estimate `eps`](figures_for_jn/f12.png)

**Figure 12**: The k-distance graph used to estimate `eps`.

![A QGIS-generated map showing anomalies detected using the DBSCAN method with flight line number, longitude, latitude, and residual magnetic field data used as inputs](figures_for_jn/f13.png)

**Figure 13**: A QGIS-generated map showing anomalies detected using the DBSCAN method with flight line number, longitude, latitude, and residual magnetic field data used as inputs